In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [2]:
DATASET_TAGS = ['PBMC1', 'PBMC2', 'PBMC3', 'PBMC4']
TOOLS = ['seurat', 'monocle', 'scanpy', 'scvi-tools', 'COTAN']
N_MARKERS = 200
RANDOM_MARKERS_TRIALS = 5
RANDOM_SEED = 42
clf = RandomForestClassifier(random_state=RANDOM_SEED)

In [3]:
feature_perc = {}
f1 = {}
num_clusters = {}

for dataset in DATASET_TAGS:
	print(f"Processing dataset {dataset}")
	feature_perc[dataset] = {}
	f1[dataset] = {}
	num_clusters[dataset] = {}
	# load gene expression matrix
	data_path = f"./data/{dataset}/filtered/10X/"
	adata = sc.read_10x_mtx(
		data_path,
		var_names='gene_symbols',
		cache=False
	)
	for tool in TOOLS:
		print(f"Tool {tool}")
		# load cluster labels
		labels_path = f"./data/{dataset}/{tool}/default/clustering_labels.csv"
		y_df = pd.read_csv(labels_path, index_col=0)
		num_clusters[dataset][tool] = np.unique(y_df['cluster']).shape[0]
		# load markers
		markers_path = f"./data/{dataset}/{tool}/default/markers.csv"
		markers_df = pd.read_csv(markers_path)
		markers = markers_df[markers_df['rank']<=N_MARKERS]['gene'].values
		markers = np.unique(markers)
		# load test cells
		test_cells_path = f"./data/{dataset}/{tool}/default/test_cells.csv"
		test_cells_df = pd.read_csv(test_cells_path)
		test_cells = test_cells_df['cell'].values
		# split data
		y_test = y_df.loc[test_cells]['cluster'].values
		y_train = y_df[~y_df.index.isin(test_cells)]['cluster'].values
		test_cells_sorted = y_df.loc[test_cells].index
		train_cells_sorted = y_df.loc[y_df.index.difference(test_cells)].index
		X_test = adata[test_cells_sorted, markers].X.toarray()
		X_train = adata[train_cells_sorted, markers].X.toarray()
		# save feature percentage
		feature_perc[dataset][tool] = X_train.shape[1]/adata.shape[1]*100
		# fit model with markers
		clf.fit(X_train, y_train)
		y_pred = clf.predict(X_test)
		# save f1 score
		f1_tool = f1_score(y_test, y_pred, average="weighted")
		f1[dataset][tool] = f1_tool
		# fit model with random features multiple times
		random_f1s = []
		for i in range(RANDOM_MARKERS_TRIALS):
			print(f"Trial {i+1}/{RANDOM_MARKERS_TRIALS}")
			# select random features
			features = np.random.choice(adata.shape[1], X_train.shape[1], replace=False)
			# split data
			X_train = adata[train_cells_sorted, features].X.toarray()
			X_test = adata[test_cells_sorted, features].X.toarray()
			# fit model
			clf.fit(X_train, y_train)
			y_pred = clf.predict(X_test)
			# save f1 score
			f1_random = f1_score(y_test, y_pred, average="weighted")
			random_f1s.append(f1_random)
		f1[dataset][f"mean random {tool} size"] = np.mean(random_f1s)
		f1[dataset][f"std random {tool} size"] = np.std(random_f1s)
		f1[dataset][f"{tool} ratio"] = f1_tool / np.mean(random_f1s)

feature_perc_df = pd.DataFrame(feature_perc)
f1_df = pd.DataFrame(f1)
feature_perc_df.to_csv("feature_perc.csv")
f1_df.to_csv("f1.csv")
print("Percentage of features used:")
display(feature_perc_df)
print("F1 scores:")
display(f1_df)
print("Number of clusters:")
display(pd.DataFrame(num_clusters))

Processing dataset PBMC1
Tool seurat


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Trial 1/5


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Trial 2/5


...

,PBMC1,PBMC2,PBMC3,PBMC4
seurat,9.035222,10.953006,12.615291,13.882353
monocle,3.765836,2.692644,4.001829,3.852941
scanpy,7.503828,7.953882,9.223264,8.220588
scvi-tools,3.480440,3.648570,3.811266,3.676471
COTAN,13.323124,16.367484,26.137663,17.455882


F1 scores:


,PBMC1,PBMC2,PBMC3,PBMC4
seurat,0.902223,0.898870,0.886936,0.833287
mean random seurat size,0.790346,0.689126,0.674554,0.698885
std random seurat size,0.016339,0.076584,0.021719,0.022526
seurat ratio,1.141554,1.304363,1.314847,1.192310
monocle,0.994457,1.000000,0.997715,0.993534
mean random monocle size,0.976546,0.979734,0.973327,0.979787
std random monocle size,0.007124,0.009691,0.021473,0.003444
monocle ratio,1.018340,1.020686,1.025056,1.014031
scanpy,0.779506,0.812081,0.790318,0.760360
mean random scanpy size,0.527751,0.590143,0.620594,0.524925


Number of clusters:


,PBMC1,PBMC2,PBMC3,PBMC4
seurat,11,14,18,19
monocle,3,2,3,3
scanpy,18,18,22,22
scvi-tools,16,16,17,16
COTAN,19,27,53,31
